在上篇博客的最后，我们说到由于效率低，[前面](../../categories/cat_sampling/)博客中介绍的方法在对高维分布进行抽样时都存在一些问题，因此我们需要一个高效可行的抽样方法。 这就引出了 MCMC，MCMC 是一系列算法的集合，包括 MH,Gibbs Sampling 和 HMC 等。它们都利用了 MCMC 这个框架来设计具体的算法实现。在介绍这些算法前，我们首先需要知道 MCMC 中的第一个 MC--马尔科夫链的性质。

想必对于计算机专业的人来说，几乎没人不知道[Page Rank](https://en.wikipedia.org/wiki/PageRank)(PR)算法。而这个算法背后的原理就是马尔科夫链。因此下面我们通过一个例子来看看PR的工作原理，同时也了解一下马尔科夫链的性质。
#### 例1:
有a,b,c,d 四个网页，其中：a 有到 a,b 的链接， b 有到 a,c 的链接，c 有到 a,b,c,d 的链接，d 有到 a,d 的链接。求 a,b,c,d 四个网页的权重。

要求得四个网页的权重，PR 的做法是，先作出a,b,c,d 的链接关系，用矩阵L表示，$L_{ij}$ 表示j有到i的链接，上面的链接关系可以得到链接矩阵：

$$
    L  = \begin{bmatrix}
    1 & 1 & 1 & 1  \\
    1 & 0 & 1 & 0  \\
    0 & 1 & 1 & 0  \\
    0 & 0 & 1 & 1  
\end{bmatrix}
$$

矩阵中第一列代表a的链接，第二列代表b的链接，以此类推。得到这个矩阵后，我们按列对这个矩阵进行归一化，表示对每个网页来说，它指向的所有链接重要性相同。归一化后得到：
$$
    A  = \begin{bmatrix}
    \frac{1}{2} & \frac{1}{2} & \frac{1}{4} & \frac{1}{2}  \\
    \frac{1}{2} & 0   & \frac{1}{4} & 0  \\
    0   & \frac{1}{2} & \frac{1}{4} & 0  \\
    0   & 0   & \frac{1}{4} & \frac{1}{2}  
\end{bmatrix}
$$
A 称作转移矩阵，用一个随机向量$\pi_0$来表示a,b,c,d四个网页的初始概率,在这个例子中我们设$\pi_0 = [0.1,0.3,0.2,0.4]^T $,PR 通过不断迭代计算$\pi_{n+1} = A\cdot\pi_n$,来得到新的$\pi_{*}$,在某次迭代后，如果$\pi_{t+1} =\pi_{t}$，则算法收敛(实践中通常利用某个收敛准则，比如两次迭代的变化量来判断算法是否收敛)。我们可以用下面的 python 代码来展示这个过程

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [23]:
pi = array([0.2,0.3,0.1,0.4])
A = array([
        [0.20, 0.3, 0.10, 0.40],
        [0.20, 0.2, 0.25, 0.25],
        [0.30, 0.3, 0.50, 0.15],
        [0.30, 0.2, 0.15, 0.20]])

def page_rank(pi,A,loop=15):
    for _ in range(loop):
        pi = np.dot(A,pi)
        print(pi)
page_rank(pi,A)


[ 0.3    0.225  0.26   0.215]
[ 0.2395   0.22375  0.31975  0.217  ]
[ 0.2338     0.2268375  0.3314     0.2079625]
[ 0.23113625  0.22696813  0.33508563  0.20681   ]
[ 0.23055025  0.22709478  0.33599563  0.20635934]
[ 0.23038178  0.22711775  0.33624522  0.20625524]
[ 0.2303383   0.22712502  0.33631076  0.20622592]
[ 0.23032661  0.22712683  0.33632826  0.20621829]
[ 0.23032352  0.22712733  0.33633291  0.20621625]
[ 0.23032269  0.22712746  0.33633414  0.20621571]
[ 0.23032247  0.22712749  0.33633447  0.20621556]
[ 0.23032241  0.2271275   0.33633456  0.20621552]
[ 0.2303224   0.2271275   0.33633458  0.20621551]
[ 0.23032239  0.2271275   0.33633459  0.20621551]
[ 0.23032239  0.22712751  0.33633459  0.20621551]


In [24]:
np.linalg.eig(A)

(array([ 1.        ,  0.26583124, -0.1       , -0.06583124]),
 array([[-0.45145539, -0.51928346, -0.48666426, -0.22199452],
        [-0.44519309,  0.08220901, -0.48666426, -0.70112805],
        [-0.65925012,  0.77892519,  0.32444284,  0.33299179],
        [-0.40420344, -0.34185074,  0.64888568,  0.59013079]]))

#### todo: 
1. 修改初始值
2. 为何收敛